In [240]:
import pandas as pd
import numpy as np
import csv

import torch
from datasets import Dataset

from torch.utils.data import DataLoader, TensorDataset
from transformers import BertForSequenceClassification, BertTokenizer, AdamW
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from sklearn.utils.class_weight import compute_class_weight





In [236]:
# Load data directly from URL (or else download it manually from patentsview.org)
#df_abst = get_stripped_data("https://s3.amazonaws.com/data.patentsview.org/download/g_patent_abstract.tsv.zip")
#df_labels = get_stripped_data("https://s3.amazonaws.com/data.patentsview.org/download/g_ipc_at_issue.tsv.zip")


In [241]:
def get_stripped_data(filename):
    data_file = pd.read_csv(
        filename,
        delimiter="\t",
        dtype=str,  # Use `dtype=str` to avoid erroneous type inference, e.g. reading `patent_id` as a number rather than a string.
        quoting=csv.QUOTE_NONE,
    )

    # Apply the strip function to each column of the DataFrame to strip the quotes
    data_file = data_file.apply(lambda col: col.map(lambda x: x.strip('"') if isinstance(x, str) else x))

    # Strips the column names of quotes
    data_file.columns = data_file.columns.str.strip('"')

    print(f"{filename} loaded and returned stripped of quotations")

    return data_file


In [238]:
# Function to return all column names one by one
def all_columns(df_name):
    for i in df_name.columns:
        print(i)

In [243]:
# load and strip the file g_patent_abstract.tsv 

df1 = get_stripped_data("g_patent_abstract.tsv")
# Remove rows where the 'text' column is empty or contains only whitespace
df1 = df1[df1['patent_abstract'].str.strip() != '']
all_columns(df1)

g_patent_abstract.tsv loaded and returned stripped of quotations
patent_id
patent_abstract


In [242]:
# load and strip the file g_ipc_at_issue.tsv 
df2 = get_stripped_data("g_ipc_at_issue.tsv")
all_columns(df2)

g_ipc_at_issue.tsv loaded and returned stripped of quotations
patent_id
ipc_sequence
classification_level
section
ipc_class
subclass
main_group
subgroup
classification_value
classification_status
classification_data_source
action_date
ipc_version_indicator


In [244]:
# print(df1.iloc[4495751])    # to look at specific patent

## IPC section classes  
  * A = Human Necessitates  
   * B = Performing Operations, Transporting  
   * C = Chemistry; Metallurgy  
  * D = Textiles; Paper  
  * E = Fixed Constructions  
  * F = Mechanical Engineering; Lighting; Heating; Weapons; Blasting  
  * G = Physics  
  * H = Electricity

 The Classification represents the whole body of knowledge which may be regarded as
proper to the field of patents for invention, divided into eight sections. Sections are the highest
level of hierarchy of the Classification.

In [245]:
# select the two columns "patent_id" and "section"
df2 = df2[["patent_id", "section"]]
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23761485 entries, 0 to 23761484
Data columns (total 2 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   patent_id  object
 1   section    object
dtypes: object(2)
memory usage: 362.6+ MB


In [246]:
df1.head(50)

,patent_id,patent_abstract
0,10000000,A frequency modulated (coherent) laser detecti...
1,10000001,The injection molding machine includes a fixed...
2,10000002,The present invention relates to: a method for...
3,10000003,The invention relates to a method for producin...
4,10000004,The present invention relates to provides a do...
5,10000005,A vacuum forming apparatus is provided that fo...
6,10000006,A thermoforming mold device (1) providing a pi...
7,10000007,An expanding tool comprising: an actuator comp...
8,10000008,"A decorated strip of coated, heat-shrinkable, ..."
9,10000009,"In sterile, additive manufacturing wherein one..."


In [247]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8177824 entries, 0 to 8979620
Data columns (total 2 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   patent_id        object
 1   patent_abstract  object
dtypes: object(2)
memory usage: 187.2+ MB


In [248]:
df1[df1["patent_id"] == "10000000"]

,patent_id,patent_abstract
0,10000000,A frequency modulated (coherent) laser detecti...


In [249]:
df2[df2["patent_id"] == "10000000"]

,patent_id,section
1982063,10000000,G
3878635,10000000,G
5536326,10000000,G
19037437,10000000,G


In [250]:
print(df1.duplicated(subset="patent_id").sum())  # Check duplicate patent_ids in df1
print(df2.duplicated(subset="patent_id").sum())  # Check duplicate patent_ids in df2 
# many duplicate patent ids in ipc at issue file (15431190)

0
15431190


In [251]:
# Check if the duplicates mean that some patents are classed with multiple sections
sections_per_patent = df2.groupby("patent_id").size()
print(sections_per_patent.describe())  # Summary statistics


count    8.330295e+06
mean     2.852418e+00
std      2.957512e+00
min      1.000000e+00
25%      1.000000e+00
50%      2.000000e+00
75%      4.000000e+00
max      2.110000e+02
dtype: float64


_Chat gpt_
#### Statistics:

* Count: 8.33 million of unique patent IDs in df2

* Mean (2.85): On average, each patent ID is associated with approximately 2.85 entries (sections).

* Standard Deviation (2.96): Indicates variability; some patents have significantly more sections associated with them than others.

* Quartiles:
        25% (1): 25% of patents are linked to only one section.
        50% (2): 50% of patents are linked to two or fewer sections (median).
        75% (4): 75% of patents are linked to four or fewer sections.
        This shows that most patents have a small number of sections.

* Maximum (211):
        The patent with the most sections is associated with 211 entries. This could indicate a special case or potential noise.

In [252]:
#ipc_sections = df2["section"].unique()

#print(f"Sections:")
#for i in ipc_sections:
#    print(i)

In [253]:
#ipc_sections.sort()
#print(ipc_sections)

In [254]:
# Look at all patent with a section class of 0
#df2[df2["section"] == "0"]

In [255]:
#print(type(df2))


In [256]:
# Define sections known and focused on
#ipc_sections_main = ["A", "B", "C", "D", "E", "F", "G", "H"]

In [257]:
# select all data with the section classes A-H
filtered_df2 = df2[df2["section"].isin(ipc_sections_main)]
print(filtered_df2.shape)

(23760193, 2)


In [258]:
# Filter out sections "A" to "H" such that I end up with a list of section classes to remove from the data
#unknown_sections = [section for section in ipc_sections if section not in ipc_sections_main]
#print(unknown_sections)


In [259]:
# Verify that only the A-H sectors are in the data now
ipc_sections = filtered_df2["section"].unique()
ipc_sections.sort()

print(f"Sections:")
for i in ipc_sections:
    print(i)

Sections:
A
B
C
D
E
F
G
H


## Take subset of data

In [260]:
# Take a random subset with determined fraction
df1_subset = df1.sample(frac=0.001, random_state=101)

# Check the shape of your subset
print(df1_subset.shape)


(8178, 2)


In [261]:
# Remove duplicates based on all columns (patent_id and section)
print(df1_subset.shape)
df1_subset = df1_subset.drop_duplicates(keep='first')
print(df1_subset.shape)

(8178, 2)
(8178, 2)


In [262]:
# Filter out all the rows with respect to patents not in the abstract dataframe (df1)
df2_subset = filtered_df2[
    filtered_df2['patent_id'].isin(df1_subset['patent_id']) &   # all patent ids in the subset of df1
    filtered_df2['section'].isin(ipc_sections_main) # which also have one of the 8 labels in ipc_sections_main (A-H) 
]
# Check the first few rows of the filtered DataFrame
print(df2_subset.head())

# You can also check the shape of the filtered DataFrame to see how many rows/columns you got
print(df2_subset.shape)


     patent_id section
1107  11480947       G
2366  10337293       E
2879  11671947       H
3011   8294150       H
4458   4736251       H
(23845, 2)


In [263]:
# Get unique rows in df2_subset
unique_rows = df2_subset.drop_duplicates()

# Print the number of unique rows
print(len(unique_rows))


10073


In [264]:
# Verify that only the A-H classes are left in the dataset
ipc_sections = df2_subset["section"].unique()
ipc_sections.sort()

print(f"Sections:")
for i in ipc_sections:
    print(i)

Sections:
A
B
C
D
E
F
G
H


In [265]:
# Remove duplicates based on all columns (patent_id and section)
# very important step since there might be duplicates where the same patent has the same main section class, but given different subclasses and therefore shows up multiple times
# combination of patent_id and section isn't unique
df2_subset = df2_subset.drop_duplicates(keep='first')

# Optionally, check the cleaned dataframe
print(df2_subset.head())
print(df2_subset.shape)

     patent_id section
1107  11480947       G
2366  10337293       E
2879  11671947       H
3011   8294150       H
4458   4736251       H
(10073, 2)


## One-Hot Encode Section Classes - really its multi-hot encoding
A patent will have a binary vector for sections, where each section index is assigned a 1 if the patent is associated with that section, and a 0 if it is not.

In [266]:
# Create one-hot encoding using get_dummies
one_hot_df = pd.get_dummies(df2_subset, columns=['section'], prefix='', prefix_sep='')
print(one_hot_df.shape)
print(one_hot_df[:10])

# Group by patent_id and sum the dummy columns to get the multiple-hot encoding
one_hot_df = one_hot_df.groupby('patent_id', as_index=False).sum()  # as_index=False is VERY important. Patent_id will become index and not considered uique values when checking for duplicates

# View the resulting one-hot encoded DataFrame
print(one_hot_df)
print(one_hot_df.shape)

(10073, 9)
     patent_id      A      B      C      D      E      F      G      H
1107  11480947  False  False  False  False  False  False   True  False
2366  10337293  False  False  False  False   True  False  False  False
2879  11671947  False  False  False  False  False  False  False   True
3011   8294150  False  False  False  False  False  False  False   True
4458   4736251  False  False  False  False  False  False  False   True
5592  11782756  False  False  False  False  False  False   True  False
5719   7237787  False   True  False  False  False  False  False  False
6799   7133578  False  False  False  False  False  False   True  False
7865   4948859  False  False   True  False  False  False  False  False
9166  11810315  False  False  False  False  False  False   True  False
     patent_id  A  B  C  D  E  F  G  H
0     10000379  0  0  1  0  0  0  0  0
1     10001403  0  0  0  0  0  0  1  0
2     10002404  0  0  0  0  0  0  1  0
3     10002897  0  0  0  0  0  0  0  1
4     1000374

In [267]:
# Check for missing values (NaN) in the dataframe
missing_values = one_hot_df.isna().sum()

print("Missing values per column:")
print(missing_values)

# If you want to see rows with missing values
print("\nRows with missing values:")
print(one_hot_df[one_hot_df.isna().any(axis=1)])

# Remove rows with any missing entries (optional after checking)
#merged_df = merged_df.dropna()

Missing values per column:
patent_id    0
A            0
B            0
C            0
D            0
E            0
F            0
G            0
H            0
dtype: int64

Rows with missing values:
Empty DataFrame
Columns: [patent_id, A, B, C, D, E, F, G, H]
Index: []


In [268]:
# Check for duplicates - none should be found
print(one_hot_df.shape)
one_hot_df = one_hot_df.drop_duplicates(keep='first')
print(one_hot_df.shape)

(8175, 9)
(8175, 9)


In [269]:
one_hot_df.head(10)

,patent_id,A,B,C,D,E,F,G,H
0,10000379,0,0,1,0,0,0,0,0
1,10001403,0,0,0,0,0,0,1,0
2,10002404,0,0,0,0,0,0,1,0
3,10002897,0,0,0,0,0,0,0,1
4,10003740,0,0,0,0,0,0,1,1
5,10004066,0,0,0,0,0,0,0,1
6,10004246,1,0,0,0,0,0,0,0
7,10006614,0,1,0,0,0,1,0,0
8,10008125,0,0,0,0,0,0,1,1
9,10008416,0,0,0,0,0,0,0,1


## Merge dataframe with abstract and dataframe with section classes

In [270]:
print(df1_subset.shape)

(8178, 2)


In [271]:
print(one_hot_df.shape)

(8175, 9)


In [272]:

# Merge one-hot-encoded sections with abstracts
merged_df = pd.merge(df1_subset, one_hot_df, on="patent_id", how="inner")   # inner merge makes sure only to take patents which both has section class AND an abstract

# Sort the merged dataframe by patent_id
merged_df = merged_df.sort_values(by="patent_id", ascending=True)

# Remove duplicates based on all columns (patent_id and section)
merged_df = merged_df.drop_duplicates(keep='first')

# Reset the index after sorting (if you want a clean index)
merged_df.reset_index(drop=True, inplace=True)

print(merged_df.head(10))
print(merged_df.shape)


  patent_id                                    patent_abstract  A  B  C  D  E  \
0  10000379  The invention is directed to a process for the...  0  0  1  0  0   
1  10001403  This invention relates to a system including a...  0  0  0  0  0   
2  10002404  A graphics processing unit (GPU) includes prog...  0  0  0  0  0   
3  10002897  Provided is a solid-state imaging device inclu...  0  0  0  0  0   
4  10003740  The present disclosure involves systems, softw...  0  0  0  0  0   
5  10004066  To handle different Quality of Service (QoS) r...  0  0  0  0  0   
6  10004246  Disclosed are hydrated fat compositions compri...  1  0  0  0  0   
7  10006614  A lighting device includes a primary housing h...  0  1  0  0  0   
8  10008125  Multi-user portable electronic devices for imp...  0  0  0  0  0   
9  10008416  A gate structure is formed over a substrate. T...  0  0  0  0  0   

   F  G  H  
0  0  0  0  
1  0  1  0  
2  0  1  0  
3  0  0  1  
4  0  1  1  
5  0  0  1  
6  0  0  0  
7  1

In [273]:
# Check for missing values (NaN) in the dataframe
missing_values = merged_df.isna().sum()

print("Missing values per column:")
print(missing_values)

# If you want to see rows with missing values
print("\nRows with missing values:")
print(merged_df[merged_df.isna().any(axis=1)])

# Remove rows with any missing entries (optional after checking)
merged_df = merged_df.dropna()

Missing values per column:
patent_id          0
patent_abstract    0
A                  0
B                  0
C                  0
D                  0
E                  0
F                  0
G                  0
H                  0
dtype: int64

Rows with missing values:
Empty DataFrame
Columns: [patent_id, patent_abstract, A, B, C, D, E, F, G, H]
Index: []


In [274]:
# Merge the columns A-H into a single column 'label_vector' with binary vectors
merged_df['label'] = merged_df[['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']].values.tolist()

merged_df['text'] = merged_df['patent_abstract']

# Select only the columns you want to keep
merged_df = merged_df[['patent_id', 'text', 'label']]


## Save merged data

In [275]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8175 entries, 0 to 8174
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   patent_id  8175 non-null   object
 1   text       8175 non-null   object
 2   label      8175 non-null   object
dtypes: object(3)
memory usage: 191.7+ KB


In [276]:
# save merged dataframes as a tsv file
merged_df.to_csv("merged_output_7.tsv", sep="\t", index=False)


In [ ]:
# Load data
#merged_df_loaded = pd.read_csv(
#        "merged_output_6.tsv",
#        delimiter="\t",
#        dtype=str,  # Use `dtype=str` to avoid erroneous type inference, e.g. inferring `patent_id` as a number rather than a string.
#    )

# Print the resulting DataFrame
#print(merged_df_loaded.head(10))

#merged_df_loaded.info()